Set base dir to sync google drive folder with colab

In [1]:
BASE_DIR = "/content/drive/MyDrive/University/Thesis/mnist_cnn_raw"
import os
os.chdir(BASE_DIR)

`save_weights_as_json.py`

In [2]:
from tensorflow import keras
import numpy as np
import json


def _kerasToList(weight: list) -> list:
    kernel = weight[0]
    bias = weight[1]
    return [kernel.numpy().tolist(), bias.numpy().tolist()]

def _saveListAsJson(listToSave: list, name: str):
    with open(name, "w") as fp:
        json.dump(listToSave, fp)
        fp.close()

def main():
    model = keras.models.load_model("./trained_model.h5", compile=True)
    model.summary()

    layer0_conv2d = model.layers[0]
    layer2_conv2d = model.layers[2]
    layer6_dense = model.layers[6]

    layer0_conv2d_weights = layer0_conv2d.weights
    layer2_conv2d_weights = layer2_conv2d.weights
    layer6_dense_weights = layer6_dense.weights

    layer0_conv2d_weights_list = _kerasToList(layer0_conv2d_weights)
    _saveListAsJson(layer0_conv2d_weights_list, "./weights/json/layer0_conv2d.json")
    layer2_conv2d_weights_list = _kerasToList(layer2_conv2d_weights)
    _saveListAsJson(layer2_conv2d_weights_list, "./weights/json/layer2_conv2d.json")
    layer6_dense_weights_list = _kerasToList(layer6_dense_weights)
    _saveListAsJson(layer6_dense_weights_list, "./weights/json/layer6_dense.json")

if __name__ == "__main__":
    main()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

`layers.py`

In [3]:
import numpy as np
import random
import math

def ReLU(image: list, shape: tuple) -> list:
    width = shape[0]
    height = shape[1]
    channelIn = shape[2]
    out = np.zeros((width, height, channelIn))
    for y in range(height):
        for x in range(width):
            for chIn in range(channelIn):
                data = image[x][y][chIn]
                out[x][y][chIn] = data * (data > 0)
    # print(f"out shape: {out.shape}")
    # print(f"in shape: {np.array(image).shape}")
    return out.tolist()

def softmax(image: list) ->list:
    imgLen = len(image)
    exps = [math.exp(pxi) for pxi in image]
    expSum = sum(exps)
    out = [ei / expSum for ei in exps]
    return out

def conv2d(inputImage: list, inputShape: tuple, kernel: list, kernelShape: tuple) -> list:
    width = inputShape[0]
    height = inputShape[1]
    channelIn = inputShape[2]
    kxSize = kernelShape[0]
    kySize = kernelShape[1]
    channelOut = kernelShape[2]

    outputImage = np.zeros((width - kxSize + 1, height - kySize + 1, channelOut))

    for y in range(height - kySize + 1):
        for x in range(width - kxSize + 1):
            for chOut in range(channelOut):
                for chIn in range(channelIn):
                    for ky in range(kySize):
                        for kx in range(kxSize):
                    
                            if ((y + ky) >= height) or ((x + kx) >= width):
                                continue
                            kr = kernel[0][kx][ky][chIn][chOut]
                            # print(f"x = {x}, y = {y}, chIn = {chIn}, kx = {kx}, ky = {ky}, chOut = {chOut}")
                            px = inputImage[x + kx][y + ky][chIn]
                            outputImage[x][y][chOut] += kr * px
                bias = kernel[1][chOut]
                outputImage[x][y][chOut] += bias 
    return outputImage.tolist()


def maxPooling2d(inputImage: list, shape: tuple, poolSize: tuple):
    width = shape[0]
    height = shape[1]
    channelIn = shape[2]
    pxSize = poolSize[0]
    pySize = poolSize[1]
    outImage = np.zeros((width // pxSize, height // pySize, channelIn))
    for y in range(0, height, pySize):
        for x in range(0, width, pxSize):
            for chIn in range(channelIn): 
                max = inputImage[x][y][chIn]
                for py in range(pySize):
                    for px in range(pxSize):
                        if (x + px) >= width or (y + py) >= height:
                            continue
                        if inputImage[x + px][y + py][chIn] > max:
                            max = inputImage[x + px][y + py][chIn]
                            if (y // pySize >= height // pySize) or \
                               (x // pxSize >= width // pxSize):
                                continue
                            outImage[x // pxSize][y // pySize][chIn] = max
    return outImage.tolist()

def flatten(inputImage: list) -> list:
    arr = np.array(inputImage)
    return arr.flatten().tolist()

def dropout(inputImageFlat: list, rate: float) -> list:
    out = np.zeros(len(inputImageFlat))
    for i in range(len(inputImageFlat)):
        if random.uniform(0, 1) < rate:
            out[i] = 0
        else:
            out[i] = inputImageFlat[i]
    # bypass layer
    return inputImageFlat

def dense(inputImageFlat: list, weights: list, shape: tuple) -> list:
    inLen = shape[0]
    outLen = shape[1]
    mulWeights = weights[0]
    biasWeights = weights[1]
    out = np.zeros(outLen)
    for o in range(outLen):
        for i in range(inLen):
            out[o] += inputImageFlat[i] * mulWeights[i][o]
        out[o] += biasWeights[o]
    return out.tolist()

`predict.py`

In [4]:
from tensorflow import keras
import numpy as np

def show1dGrayImage(imgArr, xSize, ySize, xTraverse=True):
    from PIL import Image
    img = Image.new('L', (xSize, ySize))
    if xTraverse:
        for x in range(xSize):
            for y in range(ySize):
                img.putpixel((x, y), int(imgArr[x * ySize + y] * 255))
    else:
        for x in range(xSize):
            for y in range(ySize):
                img.putpixel((x, y), int(imgArr[y * xSize + x] * 255))
    img.show()

def _loadTestData():
    # Load the data and split it between train and test sets
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
    # Scale images to the [0, 1] range
    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255
    # Make sure images have shape (28, 28, 1)
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)

    testModel = x_test[0].reshape([1, 28, 28, -1])
    return testModel

def _loadWeightsFromJson(path: str) -> list:
    with open(path, 'r') as fp:
        data = json.load(fp)
        fp.close()
    return data

def main():
    model = keras.models.load_model("./trained_model.h5", compile=True)
    layer0_conv2d_weights = _loadWeightsFromJson('./weights/json/layer0_conv2d.json')
    layer2_conv2d_weights = _loadWeightsFromJson('./weights/json/layer2_conv2d.json')
    layer6_dense_weights = _loadWeightsFromJson('./weights/json/layer6_dense.json')
    testModel = _loadTestData()
    # testModel = testModel.reshape((1, 28, 28))
    # print(model.layers[0])
    # print(testModel)
    show1dGrayImage(testModel.flatten(), 28, 28, False)
    expected = model.predict(testModel)
    print(f"expected: {expected}")

    # layer 0 -> conv2d
    # in shape = (28, 28, 1)
    # out shape = (26, 26, 32)
    layer0Out = conv2d(testModel[0], (28, 28, 1), layer0_conv2d_weights, (3, 3, 32))
    # print(f"layer0Out: {layer0Out}")

    # layer 0 activation function -> ReLU
    # in shape = (26, 26, 32)
    # out shape = (26, 26, 32)
    layer0ReLU = ReLU(layer0Out, (26, 26, 32))
    # print(f"layer0ReLU: {layer0ReLU}")

    # layer 1 -> max pooling 2d
    # in shape = (26, 26, 32)
    # out shape = (13, 13, 32)
    layer1Out = maxPooling2d(layer0ReLU, (26, 26, 32), (2, 2))
    # print(f"layer1Out: {layer1Out}")

    # layer 2 -> conv2d
    # in shape = (13, 13, 32)
    # out shape = (11, 11, 64)
    layer2Out = conv2d(layer1Out, (13, 13, 32), layer2_conv2d_weights, (3, 3, 64))
    # print(f"layer2Out: {layer2Out}")

    # layer 2 activation function -> ReLU
    # in shape = (11, 11, 64)
    # out shape = (11, 11, 64)
    layer2ReLU = ReLU(layer2Out, (11, 11, 64))
    # print(f"layer2ReLU: {layer2ReLU}")

    # layer 3 -> max pooling 2d
    # in shape = (11, 11, 64)
    # out shape = (5, 5, 64)
    layer3Out = maxPooling2d(layer2ReLU, (11, 11, 64), (2, 2))
    # print(f"layer3Out: {layer3Out}")

    # layer 4 -> flatten
    # in shape = (5, 5, 64)
    # out shape = 1600
    layer4Flatten = flatten(layer3Out)
    # print(f"layer4Flatten: {layer4Flatten}")
    
    # layer 5 -> Dropout
    # in shape = 1600
    # out shape = 1600
    layer5Dropout = dropout(layer4Flatten, 0.5)
    # print(f"layer5Dropout: {layer5Dropout}")

    # layer 6 -> Dense (fully connected)
    # in shape = 1600
    # out shape = 10
    layer6Dense = dense(layer5Dropout, layer6_dense_weights, (1600, 10))
    # print(f"layer6Dense: {layer6Dense}")

    # layer 6 activation function -> softmax
    # in shape = 10
    # out shape = 10
    layer6Softmax = softmax(layer6Dense)
    # print(f"layer6Softmax: {layer6Softmax}")

    print(f"actual: {layer6Softmax}")

if __name__ == "__main__":
    main()


1/1 [==============================] - 0s 152ms/step
expected: [[2.66833750e-10 1.66843622e-11 1.41195119e-06 3.90745163e-06
  7.59192310e-14 1.49637649e-10 1.08032766e-16 9.99994397e-01
  1.81307787e-08 2.65088858e-07]]
actual: [2.177382901536185e-09, 4.2175811089589056e-07, 8.588242303332707e-06, 5.901245541715129e-06, 5.997974092520852e-11, 2.9808442968862125e-10, 2.2218616068908406e-13, 0.9999833790087868, 4.421166111058676e-07, 1.2650929767611926e-06]


`test.py`

In [5]:
model = keras.models.load_model("./trained_model.h5", compile=True)
# modelTemp = model.layers[0]

modelTemp = keras.models.Sequential()
modelTemp.add(model.layers[0])
modelTemp.add(model.layers[1])
modelTemp.add(model.layers[2])
modelTemp.add(model.layers[3])
modelTemp.add(model.layers[4])
modelTemp.add(model.layers[5])
modelTemp.add(model.layers[6])

# expected
import tensorflow as tf
testModel = _loadTestData()
ex = modelTemp.call(tf.convert_to_tensor(testModel))[0]
print(ex.shape)

# actual
layer0_conv2d_weights = _loadWeightsFromJson('./weights/json/layer0_conv2d.json')
layer2_conv2d_weights = _loadWeightsFromJson('./weights/json/layer2_conv2d.json')
layer6_dense_weights = _loadWeightsFromJson('./weights/json/layer6_dense.json')
layer0Out = conv2d(testModel[0], (28, 28, 1), layer0_conv2d_weights, (3, 3, 32))
layer0ReLU = ReLU(layer0Out, (26, 26, 32))
layer1Out = maxPooling2d(layer0ReLU, (26, 26, 32), (2, 2))
layer2Out = conv2d(layer1Out, (13, 13, 32), layer2_conv2d_weights, (3, 3, 64))
layer2ReLU = ReLU(layer2Out, (11, 11, 64))
layer3Out = maxPooling2d(layer2ReLU, (11, 11, 64), (2, 2))
layer4Flatten = flatten(layer3Out)
layer5Dropout = dropout(layer4Flatten, 0.5)
layer6Dense = dense(layer5Dropout, layer6_dense_weights, (1600, 10))
layer6Softmax = softmax(layer6Dense)
# layer2Out = conv2d(layer1Out, (13, 13, 32), layer2_conv2d_weights, (3, 3, 64))
ac = np.array(layer6Softmax)
print(ac.shape)

(10,)
(10,)


In [6]:
count = 0
# for x in range(5):
#     for y in range(5):
#         for ch in range(64):
#             # if abs(ex[x, y, ch] - ac[x, y, ch]) > 1:
#             if ac[x, y, ch] != 0:
#                 if abs(ex[x, y, ch] - ac[x, y, ch]) > 1e-5:
#                     count += 1
#                     # print(f"{x}, {y}, {ch}, ex: {ex[x, y, ch]}, ac: {ac[x, y, ch]}, diff: {ex[x, y, ch] - ac[x, y, ch]}")
for x in range(10):
    if abs(ex[x] - ac[x]) > 0.1:
        count += 1
        print(f"{x}, ex: {ex[x]}, ac: {ac[x]}, diff: {ex[x] - ac[x]}")
print(count)

0
